In [ ]:
!pip install transformers
!pip uninstall -y torch
!pip uninstall -y torchvision
!pip install torch==1.11.0 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html
!pip install accelerate==0.20.1
!pip install transformers[torch]

Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Successfully uninstalled torch-1.11.0
Found existing installation: torchvision 0.16.0+cu121
Uninstalling torchvision-0.16.0+cu121:
  Successfully uninstalled torchvision-0.16.0+cu121
Looking in links: https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html
ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu102 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0)
ERROR: No matching distribution found for torch==1.11.0+cu102
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 879.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.1 MB/s eta 0:00:00
     ━━━

In [ ]:
import json
from transformers import pipeline

with open('train.json', 'r') as file:
    train_data = json.load(file)

with open('test.json', 'r') as file:
    test_data = json.load(file)

In [ ]:
def preprocess_data(data):
    processed_data = []

    if 'data' in data:
        paragraphs_list = data['data']
        for paragraphs_entry in paragraphs_list:
            if 'paragraphs' in paragraphs_entry:
                paragraphs = paragraphs_entry['paragraphs']
                for paragraph in paragraphs:
                    qas_list = paragraph.get('qas', [])
                    for qa in qas_list:
                        transcription = paragraph.get('context', '')
                        question = qa.get('question', '')

                        formatted_data = {
                            'context': transcription,
                            'qas': [{'question': question}]
                        }

                        processed_data.append(formatted_data)

    print("Original Training Data:", data)
    print("Processed Data:", processed_data)
    return processed_data

preprocessed_train_data = preprocess_data(train_data)

Original Training Data: {'data': [{'paragraphs': [{'qas': [{'question': ' How old is the patient?', 'id': 625398, 'answers': [{'answer_id': 676907, 'document_id': 1298870, 'question_id': 625398, 'text': '30', 'answer_start': 270, 'answer_end': 272, 'answer_category': None}], 'is_impossible': False}, {'question': 'What is the gender of the Patient', 'id': 625404, 'answers': [{'answer_id': 676908, 'document_id': 1298870, 'question_id': 625404, 'text': ' female', 'answer_start': 281, 'answer_end': 288, 'answer_category': None}], 'is_impossible': False}, {'question': 'Does the patient have any complaints?', 'id': 625399, 'answers': [{'answer_id': 676909, 'document_id': 1298870, 'question_id': 625399, 'text': 'overweight ', 'answer_start': 303, 'answer_end': 314, 'answer_category': None}], 'is_impossible': False}, {'question': 'What is the reason for this consultation?\n', 'id': 625400, 'answers': [{'answer_id': 676911, 'document_id': 1298870, 'question_id': 625400, 'text': 'gastric bypass'

In [ ]:
!pip install torch==1.11.0
!pip install accelerate -U

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, Trainer, TrainingArguments

def train_qa_model(train_data):
    if not train_data:
        raise ValueError("No training data provided.")

    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

    contexts = [entry['context'] for entry in train_data]
    questions = [qa['question'] for entry in train_data for qa in entry['qas']]

    tokenized_train_data = tokenizer(contexts, questions, return_tensors='pt', padding=True, truncation=True)

    model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')

    training_args = TrainingArguments(
        output_dir='./qa_model',
        per_device_train_batch_size=4,
        num_train_epochs=3,
        save_steps=500,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_data,
    )
    trainer.train()

    return model

  Using cached torch-1.11.0-cp310-cp310-manylinux1_x86_64.whl (750.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Successfully uninstalled torch-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.14 requires torchvision>=0.11, which is not installed.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.


In [ ]:
from transformers import TrainerCallback

class QAEvaluationCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, model, tokenizer, datasets, **kwargs):
        evaluation_results = trainer.evaluate()

        print("Evaluation Results:", evaluation_results)

evaluation_callback = QAEvaluationCallback()

def evaluate_qa_model(model, test_data):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
    tokenized_test_data = tokenizer(test_data, return_tensors='pt', padding=True, truncation=True)

    model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')

    training_args = TrainingArguments(
        output_dir='./qa_model',
        per_device_eval_batch_size=4,
        evaluation_strategy='steps',
        eval_steps=500,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=None,
        train_dataset=None,
        eval_dataset=tokenized_test_data,
        callbacks=[evaluation_callback],
    )

    return trainer.evaluate()

In [ ]:
from transformers import pipeline

def extract_information(model, transcription, questions):
    inputs = [{"question": question, "context": transcription} for question in questions]

    question_answering_pipeline = pipeline('question-answering', model=model, tokenizer=model)
    answers = question_answering_pipeline(inputs)

    extracted_information = {
        'age': answers[0]['answer'],
        'complaints': answers[1]['answer'],
        'consultation_reason': answers[2]['answer'],
        'gender': answers[3]['answer']
    }

    return extracted_information

In [ ]:
if preprocessed_train_data:
    qa_model = train_qa_model(preprocessed_train_data)
    evaluation_results = evaluate_qa_model(qa_model, test_data)
else:
    print("No training data available.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [ ]:
print(f"Exact Match: {evaluation_results['exact_match']}%")
print(f"F1 Score: {evaluation_results['f1_score']}%")

NameError: name 'evaluation_results' is not defined

In [ ]:
sample_transcription = "Patient is a 40-year-old male with complaints of chest pain."
information = extract_information(qa_model, sample_transcription)
print("Extracted Information:", information)

NameError: name 'qa_model' is not defined